<a href="https://colab.research.google.com/github/xcero/Segmentos/blob/main/Avance3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === 0. AUTENTICACIÓN GOOGLE DRIVE ===
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import geopandas as gpd
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

# === CONFIGURACIONES ===
geojson_save_path = 'segmentos.geojson'
file_id = "1EkVx2O4i617mqS11QkpC9SKp9jYlisb4"  # <-- Cambia esto al ID de tu CSV en Google Drive
arcgis_username = "marcezquivel_magsv"
arcgis_password = "Paulina@0112"
item_id = "4f8100645e654db4b500eb1716d4c563"  # Asegúrate que sea un Feature Layer hosted

# === 1. DESCARGAR CSV DESDE GOOGLE DRIVE ===
creds, _ = default()
drive_service = build('drive', 'v3', credentials=creds)

request = drive_service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)

done = False
while done is False:
    status, done = downloader.next_chunk()

fh.seek(0)
df = pd.read_csv(fh)

# === 2. VALIDAR Y CORREGIR COLUMNAS DE COORDENADAS ===
if not {'Lat', 'Long'}.issubset(df.columns):
    raise ValueError("❌ El archivo debe contener las columnas 'Lat' y 'Long'.")

df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
df['Long'] = pd.to_numeric(df['Long'], errors='coerce')

invalid_coords = df[(df['Lat'].isna()) | (df['Long'].isna()) |
                    (df['Lat'] > 90) | (df['Lat'] < -90) |
                    (df['Long'] > 180) | (df['Long'] < -180)]

if not invalid_coords.empty:
    print("⚠️ Se encontraron coordenadas inválidas:")
    print(invalid_coords[['Lat', 'Long']])
    df = df.drop(invalid_coords.index)

# === 3. CREAR Y GUARDAR GEOJSON ===
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Long'], df['Lat']), crs="EPSG:4326")
gdf.to_file(geojson_save_path, driver='GeoJSON')
print("✅ GeoJSON generado con éxito.")

# === 4. CONECTAR A ArcGIS Online ===
gis = GIS("https://www.arcgis.com", arcgis_username, arcgis_password)
item = gis.content.get(item_id)
print(f"Tipo de ítem: {item.type}")

# === 5. ACTUALIZAR EL FEATURE LAYER O GEOJSON ===
if item.type.lower() == "feature service":
    print("🔁 Sobrescribiendo Feature Layer...")
    flc = FeatureLayerCollection.fromitem(item)
    flc.manager.overwrite(geojson_save_path)
    print("✅ Feature Layer sobrescrito con éxito.")
elif item.type.lower() == "geojson":
    print("🔁 Reemplazando archivo GeoJSON...")
    item.update(data=geojson_save_path)
    print("✅ Archivo GeoJSON reemplazado con éxito.")
else:
    print(f"⚠️ Tipo de ítem no soportado para sobrescritura: {item.type}")


ValueError: A linearring requires at least 4 coordinates.